In [1]:
filelines = []

with open('./input/Lectionary 2021.txt', 'r') as txtfile:
    filelines = txtfile.readlines()
    
len(filelines)

10370

In [2]:
from enum import Enum
import re

class Section(Enum):
    UNKNOWN = 0,
    MONTH = 1
    DAY = 2,
    DAILY_EUCHARISTIC_L = 3,
    FOR_ALL_SAINTS = 4,
    MORNING_PRAYER = 5,
    EVENING_PRAYER = 6,
    REVISED_COMMON_L = 7,
    OTHER_READINGS_MORNING = 8,
    OTHER_READINGS_EVENING = 9


def isDaySection(line, currentSection):
    match = re.search("^(SUN|MON|TUE|WED|THU|FRI|SAT)\\b", line)
    if match:
        return True

    match = re.search("^[0-9]", line)
    isDayNumber = match and currentSection == Section.DAY
    return True if isDayNumber else False

def isMonthSection(line):
    match = re.search("^(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER) ", line)
    return True if match else False

def isOtherReadingsMorning(line, previousLine):
    if(re.search("^Other Readings", previousLine) and re.search("^Morning", line)):
        return True 
    return False

def isOtherReadingsEvening(line, previousLine):
    if(re.search("^Other Readings", previousLine) and re.search("^Evening", line)):
        return True 
    return False
    
def getSection(line, previousLine = None, currentSection = Section.UNKNOWN):
    if(isDaySection(line, currentSection)):
        return Section.DAY
    if(isMonthSection(line)):
        return Section.MONTH
    if(isOtherReadingsMorning(line, previousLine)):
        return Section.OTHER_READINGS_MORNING
    if(isOtherReadingsEvening(line, previousLine)):
        return Section.OTHER_READINGS_EVENING
    
    return Section.UNKNOWN
    

In [3]:
days = {}

print(isMonthSection("NOVEMBER 2020"))
print(isDaySection("2V", Section.DAY))
print(isOtherReadingsEvening("Evening", "Other Readings"))

print(getSection("NOVEMBER 2020", None))
    
    

True
True
True
Section.MONTH


In [4]:
import json

days = []

def goThroughFile():
    previousLine = None
    currentSection = Section.MONTH
    widerSection = None
    
    currentMonth = "NOVEMBER"
    currentDayOfWeek = "SAT"
    currentDay = "28"
    saints = None
    
    otherReadingsMorning = []
    otherReadingsEvening = []
    
    for line in filelines:
        
        line = line.replace('\n', '')     
        newSection = getSection(line, previousLine, currentSection)
        #print(f"{newSection} {line} ({widerSection})")
           
        hitStartOfNextDay = newSection == Section.MONTH or (newSection == Section.DAY and currentSection != Section.MONTH and currentSection != Section.DAY)
        if(hitStartOfNextDay):
            if(not currentDayOfWeek):            
                print(f"hit line {line}")
            #summarise
            dayData = {
                'date': f"{currentDayOfWeek} {currentDay} {currentMonth}",
                'dayDescription': saints,
                'otherReadingsMorning':otherReadingsMorning,
                'otherReadingsEvening':otherReadingsEvening
            }       
            days.append(dayData)
            
            #clear the data
            currentDayOfWeek = None
            currentDay = None
            otherReadingsMorning = []
            otherReadingsEvening = []      
        
        if(newSection == Section.MONTH):  
            currentMonth = line
            widerSection = None
        elif (newSection == Section.DAY and currentSection != Section.DAY):
            currentDayOfWeek = line[0:3]
            saints = line[3:]
            widerSection = Section.DAY
        elif (newSection == Section.DAY and currentSection == Section.DAY):
            match = re.search("^[0-9]+", line)
            if(match):
                currentDay = match.group(0) #the whole match
            widerSection = None
        elif (newSection == Section.OTHER_READINGS_MORNING):
            widerSection = Section.OTHER_READINGS_MORNING
        elif (newSection == Section.OTHER_READINGS_EVENING):
            widerSection = Section.OTHER_READINGS_EVENING
        elif (newSection == Section.UNKNOWN and widerSection == Section.OTHER_READINGS_MORNING and line != "Other Readings"):
            otherReadingsMorning.append(line)
        elif (newSection == Section.UNKNOWN and widerSection == Section.OTHER_READINGS_EVENING):
            otherReadingsEvening.append(line)
          
        currentSection = widerSection if widerSection else newSection
        previousLine = line
              

In [5]:
days = []
goThroughFile()

for index, value in enumerate(days):
    day = days[index]
        
    if "None" in day['date']:
        previousDay = days[index - 1]
        print(f"{day['date']} (previous day was {previousDay['date']})")
    
    

hit line SAT St Stephen, the first Christian Martyr/
hit line SUN 2nd Sunday of Christmas/Te Rātapu Tuarua i muri i te Rā Whānautanga
hit line SUN THE EPIPHANY of our Lord Jesus Christ
hit line WED ASH WEDNESDAY: The First Day of Lent/Te Wenerei Pungarehu
hit line TUE
hit line TUE Day of Prayer for the Care of Creation
hit line SUN If All Saints
None None DECEMBER 2020 (previous day was FRI 25 DECEMBER 2020)
None None JANUARY 2021 (previous day was FRI 2 JANUARY 2021)
None None JANUARY 2021 (previous day was SAT 9 JANUARY 2021)
None None FEBRUARY 2021 (previous day was TUE 16 FEBRUARY 2021)
None None AUGUST 2021 (previous day was TUE 3 AUGUST 2021)
None None OCTOBER 2021 (previous day was MON 4 OCTOBER 2021)
None None OCTOBER 2021 (previous day was SAT 30 OCTOBER 2021)


# Stop
Only continue once the above statement prints all the dates correctly (modify the text file and rerun until it is complete